### The scope of this code is to run the HMM in the parcellation ROIs and *maybe* to then compare them to the topic model outputs?

In [1]:
#### this was replaced by permuting the subjects that went into the train/test condition for the initial Event Seg
#### The script for this is called HMM_avg_node_range.py and since it takes a long time HMM_avg_node_range_fillin.py is an alternative that can be used

import warnings
warnings.filterwarnings('ignore')
import sys 
import os    
import glob
from functools import reduce
import numpy as np
from brainiak.eventseg.event import EventSegment
import nibabel as nib
from scipy.stats import zscore, norm, ttest_rel
# from nilearn.masking import apply_mask
# from nilearn.image import load_img
# from nilearn.image import resample_to_img
import pandas as pd

In [2]:
mask_dir = '/home/jovyan/paranoia_project/nh-paranoia/code/data/'

label_node = pd.read_csv(mask_dir + '_masks/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_1mm.Centroid_RAS.csv')
label_node['ROI Name']


0                  7Networks_LH_Vis_1
1                  7Networks_LH_Vis_2
2                  7Networks_LH_Vis_3
3                  7Networks_LH_Vis_4
4                  7Networks_LH_Vis_5
                   ...               
95    7Networks_RH_Default_PFCdPFCm_1
96    7Networks_RH_Default_PFCdPFCm_2
97    7Networks_RH_Default_PFCdPFCm_3
98     7Networks_RH_Default_pCunPCC_1
99     7Networks_RH_Default_pCunPCC_2
Name: ROI Name, Length: 100, dtype: object

In [11]:
pwd

'/home/jovyan/paranoia_project/nh-paranoia/code'

In [5]:
schaeffer_dir = '/home/jovyan/paranoia_project/schaeffer_dict_roi/'
save_dir = '/home/jovyan/paranoia_project/nh-paranoia/schaef_avg_events/'


node = list(label_node['ROI Name'])[0]
temp = np.load(schaeffer_dir + f'{node}_all_subs.npy',allow_pickle=True).item()
node_loaded = np.array(list(temp.values()))
n_subs, n_vox,n_ts = node_loaded.shape

In [4]:
np.reshape(node_loaded, (n_subs,n_ts,n_vox)).shape


(17, 526, 794)

In [28]:
run = 1
if run == 1:
    temp = np.load(schaeffer_dir + f'{node}_all_subs.npy',allow_pickle=True).item()
else:
    temp = np.load(schaeffer_dir + f'{node}_all_subs_run{run}.npy',allow_pickle=True).item()

node_loaded = np.array(list(temp.values()))

In [32]:
node_loaded.shape

np.concatenate([node_loaded,node_loaded],axis=2).shape

(17, 819, 1052)

In [16]:
node_loaded[:int(n_subs/2)].shape
node_loaded[int(n_subs/2):].shape

(9, 332, 819)

In [10]:
results = {}  # keys are nodes
bounds = {}
bounds_sm = {}
for node in list(label_node['ROI Name']):
    results[node] = []
    bounds[node] = []
    bounds_sm[node] = []

min_here = 5
max_here = 30
k_array = np.arange(min_here, max_here, 5)
test_ll = np.zeros(len(k_array))

run = int(input('What run do you want?'))


for node in list(label_node['ROI Name']):
    print(node)
    test_ll = np.zeros(len(k_array))
    if run == 1:
        temp = np.load(schaeffer_dir + f'{node}_all_subs.npy',allow_pickle=True).item()
    else:
        temp = np.load(schaeffer_dir + f'{node}_all_subs_run{run}.npy',allow_pickle=True).item()
        
    node_loaded = np.array(list(temp.values()))
    #temp
    print('first',node_loaded.shape)
    n_subs, n_vox,n_ts = node_loaded.shape
    node_loaded = np.reshape(node_loaded, (n_subs,n_ts,n_vox))

    ### Z-SCORING HERE TEMPORARILY IN ORDER TO REMOVE VOXELS THAT ARE BAD (SEE ABOVE)
    n_subs, n_ts, n_vox = node_loaded.shape
    print(node_loaded.shape)

    bad_vox = []
    node_z = zscore(node_loaded,axis=1)
    for sub in range(n_subs):
        for vox in range(n_vox):
            if (np.unique(np.isnan(node_z[sub, :, vox]))) == [True]: #means that they are True i.e. that they are all nans
                bad_vox.append(vox)
    bad_vox = np.unique(bad_vox) #get only the unique values
    if len(bad_vox) > 0:
        #print('bad_vox', bad_vox)
        node_loaded = np.delete(node_loaded[:, :, :],bad_vox,axis=2) #need to remove from node_loaded because do not want the z-scored
        #print(node_loaded.shape) #correctly removed the length of bad_vox! 
            
    ### training on half the data
    for i, k in enumerate(k_array):
        movie_train = np.mean(node_loaded[:int(n_subs/2)], axis=0)
        movie_HMM = EventSegment(k)
        movie_HMM.fit(movie_train)
        movie_test = np.mean(node_loaded[int(n_subs/2):], axis=0)
        _, test_ll[i] = movie_HMM.find_events(movie_test)
    max_ind = np.argmax(test_ll)

    print('Max is %d events' % k_array[max_ind])
    print('finished with part 1 for node')
    if max_ind < 6: #so that it doesn't go below 6!
        k_small = np.arange((k_array[max_ind]), (k_array[max_ind]) + 5, 1)
    else:
        k_small = np.arange((k_array[max_ind]) - 5, (k_array[max_ind]) + 5, 1)
    test_ll_small = np.zeros(len(k_small))
    for i, k in enumerate(k_small):
        movie_train = np.mean(node_loaded[:int(n_subs/2)], axis=0)
        movie_HMM = EventSegment(k)
        movie_HMM.fit(movie_train)
        movie_test = np.mean(node_loaded[int(n_subs/2):], axis=0)
        _, test_ll_small[i] = movie_HMM.find_events(movie_test)

    max_ind_fin = np.argmax(test_ll_small)
    print('Max is %d events' % k_small[max_ind_fin])
    test_ll_small[max_ind_fin]

    movie_group = np.mean(node_loaded, axis=0)
    nTRs = movie_group.shape[0]
    movie_dur = nTRs * 1  # Data acquired every 1  seconds; was 1.5 seconds for Baldassano
    

    results[node] = k_small[max_ind_fin]
    print(results[node])
    np.save(save_dir + f'events_per_roi_run{run}_HMM_avg_zscore.npy', results)

    
    HMMsm = EventSegment(n_events=k_small[max_ind_fin], split_merge=True)
    HMMsm.fit(movie_group)
    bounds_s = np.where(np.diff(np.argmax(HMMsm.segments_[0], axis=1)))[0]
    bounds_sm[node] = bounds_s
    print(bounds_s)
    np.save(save_dir + f'event_boundaries_run{run}_HMM_avg.npy', bounds)

    
    
    

What run do you want? 3


7Networks_LH_Vis_1
first (17, 794, 332)
(17, 332, 794)
Max is 5 events
finished with part 1 for node
Max is 5 events
5
[ 45 111 179 280]
7Networks_LH_Vis_2
first (17, 1149, 332)
(17, 332, 1149)
Max is 5 events
finished with part 1 for node
Max is 5 events
5
[ 43 116 187 261]
7Networks_LH_Vis_3
first (17, 947, 332)
(17, 332, 947)
Max is 5 events
finished with part 1 for node
Max is 5 events
5
[ 53 125 192 264]
7Networks_LH_Vis_4
first (17, 1590, 332)
(17, 332, 1590)
Max is 5 events
finished with part 1 for node
Max is 5 events
5
[ 57 132 203 265]
7Networks_LH_Vis_5
first (17, 946, 332)
(17, 332, 946)
Max is 20 events
finished with part 1 for node
Max is 21 events
21
[ 12  29  47  64  80  96 113 129 144 159 177 192 207 223 240 256 270 286
 301 317]
7Networks_LH_Vis_6
first (17, 1028, 332)
(17, 332, 1028)
Max is 5 events
finished with part 1 for node
Max is 6 events
6
[ 42 118 176 227 284]
7Networks_LH_Vis_7
first (17, 943, 332)
(17, 332, 943)
Max is 5 events
finished with part 1 for node

# Run ALL runs combined

In [ ]:
results = {}  # keys are nodes
bounds = {}
bounds_sm = {}
for node in list(label_node['ROI Name']):
    results[node] = []
    bounds[node] = []
    bounds_sm[node] = []

min_here = 5
max_here = 60
k_array = np.arange(min_here, max_here, 10)
test_ll = np.zeros(len(k_array))

print('RUNNING ALL RUNS')

for node in list(label_node['ROI Name']):
    print(node)
    test_ll = np.zeros(len(k_array))
    
    run_dict = {}
    for run in [1,2,3]:
        if run == 1:
            temp = np.load(schaeffer_dir + f'{node}_all_subs.npy',allow_pickle=True).item()
        else:
            temp = np.load(schaeffer_dir + f'{node}_all_subs_run{run}.npy',allow_pickle=True).item()

        node_loaded = np.array(list(temp.values()))
        #temp
        n_subs, n_vox,n_ts = node_loaded.shape
        node_loaded = np.reshape(node_loaded, (n_subs,n_ts,n_vox))
        print('reshaped node',node_loaded.shape)
        run_dict[run] = node_loaded
    node_loaded = np.concatenate([run_dict[1],run_dict[2],run_dict[3]],axis=1)

    ### Z-SCORING HERE TEMPORARILY IN ORDER TO REMOVE VOXELS THAT ARE BAD (SEE ABOVE)
    n_subs, n_ts, n_vox = node_loaded.shape
    print('concat shape',node_loaded.shape)

    bad_vox = []
    node_z = zscore(node_loaded,axis=1)
    for sub in range(n_subs):
        for vox in range(n_vox):
            if (np.unique(np.isnan(node_z[sub, :, vox]))) == [True]: #means that they are True i.e. that they are all nans
                bad_vox.append(vox)
    bad_vox = np.unique(bad_vox) #get only the unique values
    if len(bad_vox) > 0:
        #print('bad_vox', bad_vox)
        node_loaded = np.delete(node_loaded[:, :, :],bad_vox,axis=2) #need to remove from node_loaded because do not want the z-scored
        #print(node_loaded.shape) #correctly removed the length of bad_vox! 
            
    ### training on half the data
    for i, k in enumerate(k_array):
        movie_train = np.mean(node_loaded[:int(n_subs/2)], axis=0)
        movie_HMM = EventSegment(k)
        movie_HMM.fit(movie_train)
        movie_test = np.mean(node_loaded[int(n_subs/2):], axis=0)
        _, test_ll[i] = movie_HMM.find_events(movie_test)
    max_ind = np.argmax(test_ll)

    print('Max is %d events' % k_array[max_ind])
    print('finished with part 1 for node')
    if max_ind < 6: #so that it doesn't go below 6!
        k_small = np.arange((k_array[max_ind]), (k_array[max_ind]) + 5, 1)
    else:
        k_small = np.arange((k_array[max_ind]) - 5, (k_array[max_ind]) + 5, 1)
    test_ll_small = np.zeros(len(k_small))
    for i, k in enumerate(k_small):
        movie_train = np.mean(node_loaded[:int(n_subs/2)], axis=0)
        movie_HMM = EventSegment(k)
        movie_HMM.fit(movie_train)
        movie_test = np.mean(node_loaded[int(n_subs/2):], axis=0)
        _, test_ll_small[i] = movie_HMM.find_events(movie_test)

    max_ind_fin = np.argmax(test_ll_small)
    print('Max is %d events' % k_small[max_ind_fin])
    test_ll_small[max_ind_fin]

    movie_group = np.mean(node_loaded, axis=0)
    nTRs = movie_group.shape[0]
    movie_dur = nTRs * 1  # Data acquired every 1  seconds; was 1.5 seconds for Baldassano
    

    results[node] = k_small[max_ind_fin]
    print(results[node])
    run = 'all_runs_combined'
    np.save(save_dir + f'events_per_roi_run{run}_HMM_avg_zscore.npy', results)

    
    HMMsm = EventSegment(n_events=k_small[max_ind_fin], split_merge=True)
    HMMsm.fit(movie_group)
    bounds_s = np.where(np.diff(np.argmax(HMMsm.segments_[0], axis=1)))[0]
    bounds_sm[node] = bounds_s
    print(bounds_s)
    np.save(save_dir + f'event_boundaries_run{run}_HMM_avg.npy', bounds)

    
    
    

RUNNING ALL RUNS
7Networks_LH_Vis_1
reshaped node (17, 526, 794)
reshaped node (17, 452, 794)
reshaped node (17, 332, 794)
concat shape (17, 1310, 794)
Max is 5 events
finished with part 1 for node
Max is 5 events
5
[183 406 636 966]
7Networks_LH_Vis_2
reshaped node (17, 526, 1149)
reshaped node (17, 452, 1149)
reshaped node (17, 332, 1149)
concat shape (17, 1310, 1149)
Max is 5 events
finished with part 1 for node
Max is 5 events
5
[ 268  549  795 1056]
7Networks_LH_Vis_3
reshaped node (17, 526, 947)
reshaped node (17, 452, 947)
reshaped node (17, 332, 947)
concat shape (17, 1310, 947)
Max is 5 events
finished with part 1 for node
Max is 9 events
9
[  68  150  306  465  577  792 1009 1134]
7Networks_LH_Vis_4
reshaped node (17, 526, 1590)
reshaped node (17, 452, 1590)
reshaped node (17, 332, 1590)
concat shape (17, 1310, 1590)
Max is 5 events
finished with part 1 for node
Max is 5 events
5
[314 445 646 992]
7Networks_LH_Vis_5
reshaped node (17, 526, 946)
reshaped node (17, 452, 946)
re